# Overview and Imports

Recurrent Neural Networks (RNNs) are a type of neural network that are designed to handle sequential data, such as time series or natural language text. They achieve this by using a hidden state that is updated for each time step of the input sequence, allowing the network to maintain a memory of previous inputs.

This notebook contains an implementation of an RNN that can be used for language modeling. The self takes in a sequence of characters and outputs the probability distribution over the next character in the sequence. The network is trained on a corpus of text and then used to generate new text that has a similar distribution of characters as the training corpus.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

# Data Prepartion

In [7]:
class DataGenerator:
    """
    A class for generating input and output examples for a character-level language model.
    """

    def __init__(self, path):
        """
        Initializes a DataGenerator object.

        Args:
            path (str): The path to the text file containing the training data.
        """

        self.path = path

        with open(path, 'r') as f:
            data = f.read().lower()
        
        # Create list of unique characters in the data
        self.chars = list(set(data))

        # Create dictionaries mapping characters to and from their index in the list of unique characters
        self.char_to_idx = {char : index for (index, char) in enumerate(self.chars)}
        self.idx_to_char = {index: char for (index, char) in enumerate(self.chars)}

        # Set the size of the vocabulary (i.e. number of unique characters)
        self.vocab_size = len(self.chars)

        # Read in examples from file and convert to lowercase, removing leading/trailing white space
        with open(path, 'r') as f:
            examples = f.readlines()
        self.examples = [x.lower().strip() for x in examples]
    
    def generate_data(self, index):
        """
        Generates an input/output example for the language model based on the given index.

        Args:
            idx (int): The index of the example to generate.

        Returns:
            A tuple containing the input and output arrays for the example.
        """
        example_chars = self.examples[index]

        # Convert the characters in the example to their corresponding indices in the list of unique characters
        example_char_idx = [self.char_to_idx[char] for char in example_chars]

        # Add newline character as the first character in the input array, and as the last character in the output array
        X= [self.char_to_idx['\n']] + example_char_idx
        Y = example_char_idx + [self.char_to_idx['\n']]

        return np.array(X), np.array(Y)

In [9]:
data_gen = DataGenerator('../rnn/dinos.txt')
X, y = data_gen.generate_data(2)

In [10]:
X

array([19,  2, 25, 20,  2,  5,  5,  2, 22, 18,  2,  0,  3,  0, 18])

In [11]:
y

array([ 2, 25, 20,  2,  5,  5,  2, 22, 18,  2,  0,  3,  0, 18, 19])

# RNN Implementation

The RNN used in this notebook is a basic one-layer RNN. It consists of an input layer, a hidden layer, and an output layer. The input layer takes in a one-hot encoded vector representing a character in the input sequence. This vector is multiplied by a weight matrix  W_ax to produce a hidden state vector  a. The hidden state vector is then passed through a non-linear activation function (in this case, the hyperbolic tangent function) and updated for each time step of the input sequence. The updated hidden state is then multiplied by a weight matrix W_ya to produce the output probability distribution over the next character in the sequence.

The RNN is trained using stochastic gradient descent with the cross-entropy loss function. During training, the self takes in a sequence of characters and outputs the probability distribution over the next character. The true next character is then compared to the predicted probability distribution, and the parameters of the network are updated to minimize the cross-entropy loss.



## Train
The train method trains the RNN on a dataset using backpropagation through time. The method takes an instance of DataReader containing the training data as input. The method initializes a hidden state vector a_prev at the beginning of each sequence to zero. It then iterates until the smooth loss is less than a threshold value.

During each iteration, it retrieves a batch of inputs and targets from the data reader. The RNN then performs a forward pass on the input sequence and computes the output probabilities. The backward pass is performed using the targets and output probabilities to calculate the gradients of the parameters of the network. The Adagrad algorithm is used to update the weights of the network.

The method then calculates and updates the loss using the updated weights. The previous hidden state is updated for the next batch. The method prints the progress every 500 iterations by generating a sample of text using the sample method and printing the loss.

The train method can be summarized by the following steps:


1. Initialize  a_prev to zero at the beginning of each sequence.

2. Retrieve a batch of inputs and targets from the data reader.

3. Perform a forward pass on the input sequence and compute the output probabilities.

4. Perform a backward pass using the targets and output probabilities to calculate the gradients of the parameters of the network.

5. Use the Adagrad algorithm to update the weights of the network.

6. Calculate and update the loss using the updated weights.

7. Update the previous hidden state for the next batch.

8. Print progress every 10000 iterations by generating a sample of text using the sample method and printing the loss.

9. Repeat steps  2 - 8 until the smooth loss is less than the threshold value.

In [38]:
class RNN:
    """
    A class used to represent a Recurrent Neural Network (RNN).

    Attributes
    ----------
    hidden_size : int
        The number of hidden units in the RNN.
    vocab_size : int
        The size of the vocabulary used by the RNN.
    sequence_length : int
        The length of the input sequences fed to the RNN.
    learning_rate : float
        The learning rate used during training.
    is_initialized : bool
        Indicates whether the AdamW parameters has been initialized.

    Methods
    -------
    __init__(hidden_size, vocab_size, sequence_length, learning_rate)
        Initializes an instance of the RNN class.
    
    forward(self, X, a_prev)
     Computes the forward pass of the RNN.
     
    softmax(self, x)
       Computes the softmax activation function for a given input array. 
       
    backward(self,x, a, y_preds, targets)    
        Implements the backward pass of the RNN.
        
   loss(self, y_preds, targets)
     Computes the cross-entropy loss for a given sequence of predicted probabilities and true targets. 
     
    adamw(self, beta1=0.9, beta2=0.999, epsilon=1e-8, L2_reg=1e-4)
       Updates the RNN's parameters using the AdamW optimization algorithm.
       
    train(self, generated_names=5)
       Trains the RNN on a dataset using backpropagation through time (BPTT).   
       
   predict(self, start)
        Generates a sequence of characters using the trained self, starting from the given start sequence.
        The generated sequence may contain a maximum of 50 characters or a newline character.

    """

    def __init__(self, hidden_size, data_generator, sequence_length, learning_rate):
        self.hidden_size = hidden_size
        self.data_generator = data_generator
        self.vocab_size = self.data_generator.vocab_size
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate

        self.X = None
        ''' 
        Model parameters:
        W_ax: shape hidden_size * vocab_size
        W_aa: shape hidden_size * hidden_size
        W_ya: shape vocab_size * hidden_size
        ba: shape hidden_size * 1
        by: shape vocab_size * 1
        '''

        self.Wax = np.random.uniform(-np.sqrt(1/self.vocab_size), np.sqrt(1/self.vocab_size), (hidden_size, self.vocab_size))
        self.Waa = np.random.uniform(-np.sqrt(1/hidden_size), -np.sqrt(1/hidden_size), (hidden_size, hidden_size))
        self.Wya = np.random.uniform(-np.sqrt(1/hidden_size), -np.sqrt(1/hidden_size), (self.vocab_size, hidden_size))
        self.ba = np.zeros((hidden_size, 1))
        self.by = np.zeros((self.vocab_size, 1))

        # Initialize gradients
        self.dWax, self.dWaa, self.dWya = np.zeros_like(self.Wax), np.zeros_like(self.Waa), np.zeros_like(self.Wya)

        self.dba, self.dby = np.zeros_like(self.ba), np.zeros_like(self.by)

        # parameter update with AdamW
        self.mWax = np.zeros_like(self.Wax)
        self.vWax = np.zeros_like(self.Wax)
        self.mWaa = np.zeros_like(self.Waa)
        self.vWaa = np.zeros_like(self.Waa)
        self.mWya = np.zeros_like(self.Wya)
        self.vWya = np.zeros_like(self.Wya)
        self.mba = np.zeros_like(self.ba)
        self.vba = np.zeros_like(self.ba)
        self.mby = np.zeros_like(self.by)
        self.vby = np.zeros_like(self.by)
    
    def softmax(self, x):
        '''
        Computes the softmax activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the softmax activation values.
        '''
        x = x - np.max(x)
        p = np.exp(x)
        return p/np.sum(p)
    
    def forward(self, X, a_prev):
        """
        Compute the forward pass of the RNN.

        Parameters:
        X (ndarray): Input data of shape (seq_length, vocab_size)
        a_prev (ndarray): Activation of the previous time step of shape (hidden_size, 1)

        Returns:
        x (dict): Dictionary of input data of shape (seq_length, vocab_size, 1), with keys from 0 to seq_length-1
        a (dict): Dictionary of hidden activations for each time step, with keys from 0 to seq_length-1
        y_pred (dict): Dictionary of output probabilities for each time step, with keys from 0 to seq_length-1
        """

        x, a, y_pred = {}, {}, {}
        # Store the input data in the class variable for later use in the backward pass.
        self.X = X
        # Set the initial activation to the previous activation.
        a[-1] = np.copy(a_prev)
        for t in range(len(self.X)):
            # get the input at the current time step (one-hot vector)
            x[t] = np.zeros((self.vocab_size, 1))
            if self.X[t] != None:
                x[t][self.X[t]] = 1
            # get the input at the current time step
            a[t] = np.tanh(np.dot(self.Wax, x[t]) + np.dot(self.Waa, a[t-1]) + self.ba)

            y_pred[t] = self.softmax(np.dot(self.Wya, a[t]) + self.by)
        
        return x, a, y_pred
    
    def backward(self, x, a, y_pred, targets):
        """
        Implement the backward pass of the RNN.

        Args:
        x -- (dict) of input characters (as one-hot encoding vectors) for each time-step, shape (vocab_size, sequence_length)
        a -- (dict) of hidden state vectors for each time-step, shape (hidden_size, sequence_length)
        y_preds -- (dict) of output probability vectors (after softmax) for each time-step, shape (vocab_size, sequence_length)
        targets -- (list) of integer target characters (indices of characters in the vocabulary) for each time-step, shape (1, sequence_length)

        Returns:
        None

        """

        # Initialize derivative of hidden state for the last time-step
        da_next = np.zeros_like(a[0])

        # Loop through the input sequence backwards
        for t in reversed(range(len(self.X))):
            dy_preds = np.copy(y_pred[t])
            dy_preds[targets[t]] -= 1

            da = np.dot(self.Wya.T, dy_preds) + np.dot(self.Waa.T, da_next)
            dtanh = 1 - np.power(a[t],2)
            da_unactivated = dtanh*da

            self.dba += da_unactivated
            self.Wax += np.dot(da_unactivated, x[t].T)
            self.dWaa += np.dot(da_unactivated, a[t-1].T)

            da_next = da_unactivated

            self.dWya += np.dot(dy_preds, a[t].T)

            # clip gradients to avoid exploding gradients
            for grad in [self.dWax, self.dWaa, self.dWya, self.dba, self.dby]:
                np.clip(grad, -1, 1, out=grad)
    
    def loss(self, y_preds, targets):
        """
        Computes the cross-entropy loss for a given sequence of predicted probabilities and true targets.

        Parameters:
            y_preds (ndarray): Array of shape (sequence_length, vocab_size) containing the predicted probabilities for each time step.
            targets (ndarray): Array of shape (sequence_length, 1) containing the true targets for each time step.

        Returns:
            float: Cross-entropy loss.
        """

        return sum(-np.log(y_preds[t][targets[t], 0]) for t in range(len(self.X)))
    
    def adamW(self, beta1=0.9, beta2=0.999, epsilon=1e-8, L2_reg=1e-4):
        """
        Updates the RNN's parameters using the AdamW optimization algorithm.
        """
        # AdamW update for Wax
        self.mWax = beta1 * self.mWax + (1 - beta1) * self.dWax
        self.vWax = beta2 * self.vWax + (1 - beta2) * np.square(self.dWax)
        m_hat = self.mWax / (1 - beta1)
        v_hat = self.vWax / (1 - beta2)
        self.Wax -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wax)

        # AdamW update for Waa
        self.mWaa = beta1 * self.mWaa + (1 - beta1) * self.dWaa
        self.vWaa = beta2 * self.vWaa + (1 - beta2) * np.square(self.dWaa)
        m_hat = self.mWaa / (1 - beta1)
        v_hat = self.vWaa / (1 - beta2)
        self.Waa -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Waa)

        # AdamW update for Wya
        self.mWya = beta1 * self.mWya + (1 - beta1) * self.dWya
        self.vWya = beta2 * self.vWya + (1 - beta2) * np.square(self.dWya)
        m_hat = self.mWya / (1 - beta1)
        v_hat = self.vWya / (1 - beta2)
        self.Wya -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wya)

        # AdamW update for ba
        self.mba = beta1 * self.mba + (1 - beta1) * self.dba
        self.vba = beta2 * self.vba + (1 - beta2) * np.square(self.dba)
        m_hat = self.mba / (1 - beta1)
        v_hat = self.vba / (1 - beta2)
        self.ba -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.ba)

        # AdamW update for by
        self.mby = beta1 * self.mby + (1 - beta1) * self.dby
        self.vby = beta2 * self.vby + (1 - beta2) * np.square(self.dby)

    def sample(self):
        """
        Sample a sequence of characters from the RNN.

        Args:
            None

        Returns:
            list: A list of integers representing the generated sequence.
        """

        # initialize input and hidden state
        x = np.zeros((self.vocab_size, 1))
        a_prev = np.zeros((self.hidden_size, 1)) 

        # create an empty list to store the generated character indices
        indices = []

        # idx is a flag to detect a newline character, initialize it to -1
        idx = -1

        # generate sequence of characters
        counter = 0
        max_chars = 50 # maximum number of characters to generate
        newline_character = self.data_generator.char_to_idx['\n'] # the newline character

        while (idx != newline_character and counter != max_chars):
            # compute the hidden state
            a = np.tanh(np.dot(self.Wax, x) + np.dot(self.Waa, a_prev) + self.ba)

            # compute the output probabilities
            y = self.softmax(np.dot(self.Wya, a) + self.by)

            # sample the next character from the output probabilities
            idx = np.random.choice(list(range(self.vocab_size)), p=y.ravel())

            # set the input for the next time step
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1

            # store the sampled character index in the list
            indices.append(idx)

            # update the previous hidden state
            a_prev = a

            # increment the counter
            counter += 1

        # return the list of sampled character indices
        return indices 
    def train(self, generated_names=5):
        """
        Train the RNN on a dataset using backpropagation through time (BPTT).

        Args:
        - generated_names: an integer indicating how many example names to generate during training.

        Returns:
        - None
        """

        iter_num = 0
        threshold = 5 # stopping criterion for training
        smooth_loss = -np.log(1.0 / self.data_generator.vocab_size) * self.sequence_length  # initialize loss

        while (smooth_loss > threshold):
            a_prev = np.zeros((self.hidden_size, 1))
            idx = iter_num % self.vocab_size
            # get a batch of inputs and targets
            inputs, targets = self.data_generator.generate_data(idx)

            # forward pass
            x, a, y_pred  = self.forward(inputs, a_prev)

            # backward pass
            self.backward(x, a, y_pred, targets)

            # calculate and update loss
            loss = self.loss(y_pred, targets)
            self.adamW()
            smooth_loss = smooth_loss * 0.999 + loss * 0.001

            # update previous hidden state for the next batch
            a_prev = a[len(self.X) - 1]
            # print progress every 500 iterations
            if iter_num % 500 == 0:
                print("\n\niter :%d, loss:%f\n" % (iter_num, smooth_loss))
                for i in range(generated_names):
                    sample_idx = self.sample()
                    txt = ''.join(self.data_generator.idx_to_char[idx] for idx in sample_idx)
                    txt = txt.title()  # capitalize first character 
                    print ('%s' % (txt, ), end='')
            iter_num += 1

    def predict(self, start):
        """
        Generate a sequence of characters using the trained self, starting from the given start sequence.
        The generated sequence may contain a maximum of 50 characters or a newline character.

        Args:
        - start: a string containing the start sequence

        Returns:
        - txt: a string containing the generated sequence
        """

        # Initialize input vector and previous hidden state
        x = np.zeros((self.vocab_size, 1))
        a_prev = np.zeros((self.hidden_size, 1))

        # Convert start sequence to indices
        chars = [ch for ch in start]
        idxes = []
        for i in range(len(chars)):
            idx = self.data_generator.char_to_idx[chars[i]]
            x[idx] = 1
            idxes.append(idx)

        # Generate sequence
        max_chars = 50  # maximum number of characters to generate
        newline_character = self.data_generator.char_to_idx['\n']  # the newline character
        counter = 0
        while (idx != newline_character and counter != max_chars):
            # Compute next hidden state and predicted character
            a = np.tanh(np.dot(self.Wax, x) + np.dot(self.Waa, a_prev) + self.ba)
            y_pred = self.softmax(np.dot(self.Wya, a) + self.by)
            idx = np.random.choice(range(self.vocab_size), p=y_pred.ravel())

            # Update input vector, previous hidden state, and indices
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1
            a_prev = a
            idxes.append(idx)
            counter += 1

        # Convert indices to characters and concatenate into a string
        txt = ''.join(self.data_generator.idx_to_char[i] for i in idxes)

        # Remove newline character if it exists at the end of the generated sequence
        if txt[-1] == '\n':
            txt = txt[:-1]

        return txt

In [39]:
data_generator = DataGenerator('/home/tiendat/AI-LAB/NLP/pytorch/rnn/dinos.txt')
rnn = RNN(hidden_size=200,data_generator=data_generator, sequence_length=25, learning_rate=1e-3)
rnn.train()



iter :0, loss:82.359667

Uwphcdiyxvpknw
Spjuoxjozvswhvilkqkgbeje
Lrds
Fimgvratoulfmhpjlfmcyqnheromznaxtujtiblsbkvtyvvwr
Ujkcqeazfdojavkfuszptwxoobhwezerpembsdrnrzvqywdbdv

iter :500, loss:67.332526

Eabymautuasavjdvoamacohjdpiwnruzjai
Yrlcgqoruogqtdqwlzocnjmzhqhmuafwfkmjbr
Nzcazxukmodahacrzxvamr
Drgcvppqrahqi
Mmcabowcxjujykmshabadxxaxahzgpxavamjbaqrnasrnkix


iter :1000, loss:57.525419

Ihnvxaifnauautioph
MguacejrqlgsgzsrzefaqyxhyafqmzvqmfnajhifbqdqlfwaedG
Timop
Mv


iter :1500, loss:51.502278

CqveuwygjmpaftnhskishakacpipuobpeacfuqwabamamynwikUvsqya
IasrgpierrwtbwprtwlaxmuaujmrwacwbacrboljlkgucjecgaQamrmanzcrultavbytoyjkbaufrqdnmpbrvcbofkyamjvwsveaZanzesia


iter :2000, loss:47.837806

Zswyiycquvlauj
ZdcxiesakackzgzcqfhaxncrqhbnktqschtafcsadxqphgntuoXpkxkgwwsyulmwjxwrixkpiguxfxnqcnfwq
RavayaqazddgsasadpbbirfacxeaiauxzdwooxbdtrnpxovojaTodafcjlzgmgkakgqfctbrkjmohfcgnavstawgywcojofaqrha

iter :2500, loss:45.584445

RnbymwmasacohqbtbbkdxrdhnqzctrxgechbeclyzfbazlljtaXxwdhkeabvbmnmcedalaj